In [1]:
import pandas as pd
import numpy as np
import glob
import json
import os
import sys
from tqdm import tqdm
sys.path.insert(0, '../')
# from utilities import *
from sklearn.metrics.pairwise import cosine_similarity
import torch

In [2]:
# !pip install nltk

In [3]:
# import nltk
# nltk.download("all")

In [4]:
# !pip install transformers

In [2]:
from nltk import tokenize
import nltk
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import torch.nn.functional as F

In [3]:
def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)
    return sents


In [7]:
# import os
# import glob
# import pandas as pd


# # Get a list of all the files in the annual reports folder
# annual_reports_pattern = os.path.join(annual_reports_folder, "*.txt")
# annual_reports_list = glob.glob(annual_reports_pattern)

# # Get a list of all the files in the gold summaries folder
# gold_summaries_pattern = os.path.join(gold_summaries_folder, "*.txt")
# gold_summaries_list = glob.glob(gold_summaries_pattern)

# # Sort the lists to maintain order
# annual_reports_list.sort()
# gold_summaries_list.sort()

# # Extract just the file names from the full paths
# annual_reports_list = [os.path.basename(file_path) for file_path in annual_reports_list]
# gold_summaries_list = [os.path.basename(file_path) for file_path in gold_summaries_list]

# # Create a DataFrame with "Input" and "Output" columns
# data = {
#     "Input": annual_reports_list,
#     "Output": gold_summaries_list
# }
# dataset = pd.DataFrame(data)

# # Now, 'dataset' contains two columns: "Input" and "Output" with sorted file paths
# dataset


In [4]:
import os
import glob
import pandas as pd


# Folder paths for annual reports and gold summaries
annual_reports_folder = r"/kaggle/input/annualreports/training/training/annual_reports"
gold_summaries_folder = r"/kaggle/input/combined-data"


# Get a list of all the files in the annual reports folder
annual_reports_pattern = os.path.join(annual_reports_folder, "*.txt")
annual_reports_list = glob.glob(annual_reports_pattern)

# Get a list of all the files in the gold summaries folder
gold_summaries_pattern = os.path.join(gold_summaries_folder, "*.txt")
gold_summaries_list = glob.glob(gold_summaries_pattern)

# Sort the lists to maintain order
annual_reports_list.sort()
gold_summaries_list.sort()

# Create a DataFrame with "Input" and "Output" columns
data = {
    "Input": annual_reports_list,
    "Output": gold_summaries_list
}
dataset = pd.DataFrame(data)

# Now, 'dataset' contains two columns: "Input" and "Output" with sorted file paths

# dataset = dataset[:50]

In [ ]:
# Function to count the number of words in a text file
def count_words_in_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            words = text.split()
            return len(words)
    except Exception as e:
        print(f"Error counting words in {file_path}: {str(e)}")
        return None

# Add a new column for the length of annual reports
dataset['AnnualReportLength'] = dataset['Input'].apply(count_words_in_file)

# Add a new column for the length of gold summaries
dataset['GoldSummaryLength'] = dataset['Output'].apply(count_words_in_file)

# Display the updated dataset
# print(dataset)


In [ ]:
dataset

In [5]:
# dataset.to_excel('xyz.xlsx', index=False)



In [ ]:
import pandas as pd

# Assuming you already have the 'dataset' DataFrame as mentioned in the previous code

# Initialize lists to store the contents of input and output files
input_contents = []
output_contents = []

# Iterate through the rows of the dataset and read file contents
for index, row in dataset.iterrows():
    input_file_path = row["Input"]
    output_file_path = row["Output"]
    
    # Read the content of the input file
    with open(input_file_path, "r") as input_file:
        input_content = input_file.read()
        input_contents.append(input_content)
    
    # Read the content of the output file
    with open(output_file_path, "r") as output_file:
        output_content = output_file.read()
        output_contents.append(output_content)

# Now, 'input_contents' and 'output_contents' contain the contents of input and output files, respectively
# You can access the contents using list indexing, e.g., input_contents[0], output_contents[0]
combined_df = pd.DataFrame({'Input_Content': input_contents, 'Output_Content': output_contents})
combined_df

In [ ]:
combined_df["Input_Content"][0]

In [12]:
# import pandas as pd

# # Function to count the number of words in a text file
# def count_words_in_file(file_path):
#     try:
#         with open(file_path, 'r', encoding='utf-8') as file:
#             text = file.read()
#             words = text.split()
#             return len(words)
#     except Exception as e:
#         print(f"Error counting words in {file_path}: {str(e)}")
#         return None

# # Function to count the number of characters in a text file
# def count_characters_in_file(file_path):
#     try:
#         with open(file_path, 'r', encoding='utf-8') as file:
#             text = file.read()
#             return len(text)
#     except Exception as e:
#         print(f"Error counting characters in {file_path}: {str(e)}")
#         return None

# # Add a new column for the length of annual reports (words and characters)
# dataset['AnnualReportWordCount'] = dataset['Input'].apply(count_words_in_file)
# dataset['AnnualReportCharacterCount'] = dataset['Input'].apply(count_characters_in_file)

# # Add a new column for the length of gold summaries (words and characters)
# dataset['GoldSummaryWordCount'] = dataset['Output'].apply(count_words_in_file)
# dataset['GoldSummaryCharacterCount'] = dataset['Output'].apply(count_characters_in_file)

# # Define the path for the Excel file where you want to save the DataFrame
# excel_file_path = "/kaggle/working/dataset_with_counts.xlsx"

# # Export the updated DataFrame to an Excel file
# dataset.to_excel(excel_file_path, index=False)

# # Print a message to confirm the export
# print(f"DataFrame with word and character counts has been exported to {excel_file_path}")


DataFrame with word and character counts has been exported to /kaggle/working/dataset_with_counts.xlsx


In [ ]:
pip install --upgrade pyLDAvis


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import re
import numpy as np
import pandas as pd
import time

import nltk
import pyLDAvis
import pyLDAvis.gensim_models
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from matplotlib import pyplot as plt

import gensim
import string
from gensim import corpora
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import multiprocessing
num_cores = multiprocessing.cpu_count()

# Check versions to ensure that there are no compatability issues:
python_version = !python --version
print("Python Version: ", python_version)
print("Current Directory: ", os.getcwd())
print("Numpy version: ", np.__version__)
print("Pandas version: ", pd.__version__)

In [10]:
# import os
# import glob
# import pandas as pd

# # Folder paths for annual reports and gold summaries
# annual_reports_folder = r"/kaggle/input/annualreports/training/training/annual_reports"
# gold_summaries_folder = r"/kaggle/input/combined-data"

# # Get a list of all the files in the annual reports folder
# annual_reports_pattern = os.path.join(annual_reports_folder, "*.txt")
# annual_reports_list = glob.glob(annual_reports_pattern)

# # Get a list of all the files in the gold summaries folder
# gold_summaries_pattern = os.path.join(gold_summaries_folder, "*.txt")
# gold_summaries_list = glob.glob(gold_summaries_pattern)

# # Sort the lists to maintain order
# annual_reports_list.sort()
# gold_summaries_list.sort()

# # Initialize lists to store the contents of input and output files
# input_contents = []
# output_contents = []

# # Read the contents of the annual reports files
# for annual_report_file in annual_reports_list:
#     with open(annual_report_file, "r") as input_file:
#         input_content = input_file.read()
#         input_contents.append(input_content)

# # Read the contents of the gold summaries files
# for gold_summary_file in gold_summaries_list:
#     with open(gold_summary_file, "r") as output_file:
#         output_content = output_file.read()
#         output_contents.append(output_content)

# # Now, 'input_contents' and 'output_contents' contain the contents of input and output files, respectively
# # You can access the contents using list indexing, e.g., input_contents[0], output_contents[0]


In [ ]:
len(input_contents[0]), len(output_contents[0])

In [12]:
# input_contents[0]

In [ ]:
def remove_non_ascii(text_list):
    clean_list = []
    for text in text_list:
        try:
            temp_encode = text.encode('ascii', 'ignore')
            temp_decode = temp_encode.decode('ascii')
        except:
            continue
        finally:
            clean_list.append(temp_decode)
    return clean_list

In [ ]:
# combined_df = remove_non_ascii(combined_df)

In [ ]:
unclean_full_text = list(combined_df['Input_Content'])
clean_full_text = remove_non_ascii(unclean_full_text)

In [ ]:
unclean_summary_text = list(combined_df['Output_Content'])
clean_summary_text = remove_non_ascii(unclean_summary_text)

In [17]:
# clean_full_text[0]

In [ ]:
combined_df["Input_Content"] = clean_full_text
combined_df["Output_Content"] = clean_summary_text

In [ ]:
combined_df.head()

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# def remove_stopwords(tokenized_word_list):
#     clean_list = [w for w in tokenized_word_list if not w in stop_words]
#     return clean_list

# def clean_text(test_string):
#     new_string = re.sub(r'[!"#%&\'()*+,-.\/:;<=>?@\[\\\]^_`{\|}~]|[\d]+',"", test_string)
#     new_string2 = re.sub(r'\w{3}.[A-Z|a-z]+.com', "", new_string)
#     new_string3 = re.sub(r'\S+@\S+', '', new_string2)
#     new_string4 = re.sub(r'http\S+', '', new_string3)
#     return new_string4
# import re

# def clean_text2(text):
#     # Define a regex pattern to remove illegal characters
#     illegal_char_pattern = re.compile(r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F-\x9F]')
    
#     # Use regex to remove illegal characters
#     cleaned_text = re.sub(illegal_char_pattern, '', text)
    
#     return cleaned_text


# # def tokenize_text(test_string):
# #     tokenizer = RegexpTokenizer(r'\w+|$[0-9.]+|\S+')
# #     tokenized_string = tokenizer.tokenize(test_string)
# #     return tokenized_string

# # def get_LDA_model(text_data, num_topics, passes):
# #     start_time = time.time()
# #     dictionary = corpora.Dictionary(text_data)
# #     corpus = [dictionary.doc2bow(text) for text in text_data]
   
# #     LDA_model = gensim.models.ldamulticore.LdaMulticore(
# #                                corpus=corpus,
# #                                num_topics=num_topics,
# #                                id2word=dictionary,
# #                                workers=6,
# #                                passes=passes)
    
# #     topics = LDA_model.print_topics(num_words=10)
# #     for x in topics:
# #         print(x)
# #         print("\n")
# #     print("Time Taken: ", time.time() - start_time)
# #     return LDA_model, topics, corpus, dictionary

In [ ]:
import re

def clean_text2(text):
    # Check if the input is a string
    if not isinstance(text, str):
        return text
    
    # Define a regex pattern to remove illegal characters
    illegal_char_pattern = re.compile(r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F-\x9F]')
    
    # Use regex to remove illegal characters
    cleaned_text = re.sub(illegal_char_pattern, '', text)
    
    return cleaned_text
combined_df['Input_Content'] = combined_df['Input_Content'].apply(clean_text2)
combined_df['Output_Content'] = combined_df['Output_Content'].apply(clean_text2)

In [23]:
# combined_df.to_csv("Training_Data_Clean.csv")

In [ ]:
combined_df.rename(columns={'Input_Content': 'ctext', 'Output_Content':'text'}, inplace=True)
combined_df.head(10)

In [ ]:
combined_df['text'][0]

In [ ]:
# Remove both "/" and "\n" characters from the 'Output_Content' column
combined_df['text'] = combined_df['text'].str.replace(r'[/\n]', ' ', regex=True)
# Remove both "/" and "\n" characters from the 'Output_Content' column
combined_df['ctext'] = combined_df['ctext'].str.replace(r'[/\n]', ' ', regex=True)


In [ ]:
combined_df

In [ ]:
combined_df.to_csv("Training_Data_Clean.csv")